In [1]:
import glob
import lizard
import json
import csv
import os
import re
import pickle
from os import path
from prep_for_headers import prep
from prep_for_headers import prep_for_macros
from getusefullheaders import find_all_used_header 
from comment_remover import comment_remover

In [4]:
cnt = 0
lenn = 0
pickle_list = []
lenlol = []

while True:
    try:
        try:
            with open('pickle_list.pkl', 'rb') as f:
                pickle_list = pickle.load(f)
                cnt = len(pickle_list)
        except:
            pass

        for file in glob.glob("*.cpp"):
            lenn = lenn + 1
        if lenn == len(pickle_list):
            break
        if len(pickle_list) > 0:
            mode = 'a'
        else:
            mode = 'w'
        with open("out.json",mode) as file_handler:
            if len(pickle_list) == 0:
                file_handler.write("[")
            ct = 1
            for file in glob.glob("*.cpp"):
                if file in pickle_list:
                    continue
                cnt = cnt + 1
                print(cnt, file)
                i = lizard.analyze_file(file)
                ob = i.__dict__
                if len(ob['function_list']) == 0:
                    continue
                #print (i.function_list[0].__dict__)
                json_data=open('./' + file.partition(".")[0] + '.json')
                data = json.load(json_data)
                json_data.close()
                d = {}
                d["filename"] = ob['filename']
                d["nloc"] = ob['nloc']
                d["token_count"] = ob['token_count']
                d["problem"] = data['problemlink']
                d["solution"] = data['sollink']
                d["exec_time"] = ' '.join(data['exec_time'].split())
                d["memory"] = ' '.join(data['memory'].split())
                d["usertype"] = data["usertype"]

                d["function_count"] = len(ob['function_list'])
                c = 0
                mc = 0
                used_headers = 0
                used_macros = 0;
                with open(file, 'r') as f:
                    #f.sub('//.*?\n|/\*.*?\*/', '', text, flags=re.S)
                    #print(f.read())
                    f = comment_remover(f.read())

                    header_list = prep(f)
                    #print(header_list)
                    macro_list = prep_for_macros(f)
                    x = find_all_used_header(f,header_list[0],header_list[1])
                    #print(x)
                    if len(macro_list[0]) > 0:
                        #print(macro_list[0])
                        #print(macro_list[1])
                        macrox = find_all_used_header(f,macro_list[0],macro_list[1])
                        #print(macrox)
                        for key, val in macrox.items():
                            if val == 1:
                                used_macros += 1

                    for key, val in x.items():
                        if val == 1:
                            used_headers += 1

                    for line in f.split('\n'):
                        #line = line.strip()
                        if '#include' in line:
                            #print(line)
                            c+=1
                        if '#define' in line or 'typedef' in line:
                            #print(line)
                            mc+=1
                #Take c value here
                d["library_count"] = c
                d["used_headers"] = used_headers
                #print(mc)
                d["macro_count"] = mc
                d["used_macros"] = used_macros
                c = 0
                fd = []
                ct = 0
                for func in ob['function_list']:
                    fn = i.function_list[ct].__dict__
                    ct = ct + 1
                    fd.append(fn)
                d["function_details"] = fd
                if(len(d['usertype']) == 2):
                    d['usertype'] = 'us'
                item = json.dumps(d, ensure_ascii=False)
                #print(jsonob)
                try:
                    if cnt < lenn:
                        file_handler.write("{},\n".format(item))
                    else:
                        file_handler.write("{}\n".format(item))
                except:
                    print(len(d['usertype']))
                    exit(-1)
                pickle_list.append(file)
            file_handler.write("]") 
            with open('pickle_list.pkl', 'wb') as f:
                pickle.dump(pickle_list, f)
            break
    except Exception as e:
        pickle_list.append(file)
        if str(e) == 'len(headers) != len(positions but expected to be of same length':
            lenlol.append(file)
        with open('pickle_list.pkl', 'wb') as f:
            pickle.dump(pickle_list, f)


1 1.cpp
2 2.cpp
3 3.cpp


In [3]:
f[26:35]

'#define f'

In [6]:
file

'2072.cpp'

In [3]:
lenlol

['1542574036.cpp',
 '1542577060.cpp',
 '1542578670.cpp',
 '1542578762.cpp',
 '1542578872.cpp',
 '1542580978.cpp',
 '1542584292.cpp',
 '1542587500.cpp',
 '1542588089.cpp',
 '1542589932.cpp',
 '1542592484.cpp',
 '1542593239.cpp',
 '1542594570.cpp']